In [ ]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.keras.layers import Input, concatenate, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, UpSampling2D, ZeroPadding2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from skimage.transform import resize
import os
import csv
import PIL
import numpy as np
import random
import cv2
import imutils
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2

In [ ]:
HEIGHT = 240
WIDTH = 320
INIT_LR = 0.0001
EPOCHS = 15
TRAIN_PATH = "./data/nyu2_train.csv"
TEST_PATH = "./data/nyu2_test.csv"

In [ ]:
def downsampling_block(input_tensor, n_filters):
  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(input_tensor)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

def upsampling_block(input_tensor, n_filters, name, concat_with):
  x = UpSampling2D((2, 2), interpolation='bilinear', name=name)(input_tensor)
  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convA")(x)
  x = LeakyReLU(alpha=0.2)(x)

  x = concatenate([x, concat_with], axis=3)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convB")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convC")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

In [ ]:
def build(height, width, depth):
  # input
  i = Input(shape=(height, width, depth))

  iresnet = InceptionResNetV2(include_top = False, weights = "imagenet", input_tensor = i)
  iresnet.summary()


  conv1 = iresnet.get_layer("input_layer").output
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = iresnet.get_layer("activation").output
  conv2 = ZeroPadding2D(((1,0),(1,0)))(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = iresnet.get_layer("activation_3").output
  conv3 = ZeroPadding2D((1,1))(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = iresnet.get_layer("activation_74").output
  conv4 = ZeroPadding2D(((2,1),(2,1)))(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)


  # bottleneck

  conv5 = iresnet.get_layer("activation_161").output
  conv5 = ZeroPadding2D((1,1))(conv5)
  conv5 = LeakyReLU(alpha=0.2)(conv5)

  print(conv5.shape,conv4.shape)

  # decoder
  conv6 = upsampling_block(conv5, 256, "up1", concat_with=conv4)
  conv7 = upsampling_block(conv6, 128, "up2", concat_with=conv3)
  conv8 = upsampling_block(conv7, 64, "up3", concat_with=conv2)
  conv9 = upsampling_block(conv8, 32, "up4", concat_with=conv1)

  # output
  o = Conv2D(filters=1, kernel_size=3, strides=(1,1), activation='sigmoid', padding='same', name='conv10')(conv9)

  model = Model(inputs=i, outputs=o)
  return model

In [ ]:
model = build(HEIGHT, WIDTH, 3)

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


Model: "inception_resnet_v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 127, 127, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 127, 127, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 127, 127, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 125, 125, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 125, 125, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 125, 125, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 125, 125, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 125, 125, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 125, 125, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 62, 62, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 62, 62, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 62, 62, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 62, 62, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 60, 60, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 60, 60, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 54,336,736 (207.28 MB)

 Trainable params: 54,276,192 (207.05 MB)

 Non-trainable params: 60,544 (236.50 KB)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


(None, 16, 16, 288) (None, 32, 32, 256)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 127, 127, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 127, 127, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 127, 127, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 125, 125, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 125, 125, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 125, 125, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 125, 125, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 125, 125, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 125, 125, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 62, 62, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 62, 62, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 62, 62, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 62, 62, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 60, 60, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 60, 60, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 31,159,265 (118.86 MB)

 Trainable params: 31,120,161 (118.71 MB)

 Non-trainable params: 39,104 (152.75 KB)

In [ ]:
# download nyu depth v2 dataset from gitlab
!git clone https://gitlab.com/siddinc/new_depth.git ./data

Cloning into './data'...
remote: Enumerating objects: 65371, done.
remote: Counting objects: 100% (65371/65371), done.
remote: Compressing objects: 100% (65369/65369), done.
remote: Total 65371 (delta 2), reused 65371 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (65371/65371), 2.79 GiB | 57.26 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (65418/65418), done.


In [ ]:
# loading the dataset

def read_csv(csv_file_path):
  with open(csv_file_path, 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    return [('./' + row[0], './' + row[1]) for row in csv_reader if len(row) > 0]

def train_val_split(train_paths, val_size):
  random.shuffle(train_paths)
  len_train_paths = len(train_paths)
  i = int(len_train_paths*(1.0 - val_size))
  train = train_paths[0:i]
  val = train_paths[i:len(train_paths)]
  return train, val

def load_train_paths(train_path):
  train_paths = read_csv(train_path)
  labels = {img_path: dm_path for img_path, dm_path in train_paths}
  x_paths = [img_path for img_path, dm in train_paths]
  x_train_paths, x_val_paths = train_val_split(x_paths, 0.3)

  partition = {
    'train': x_train_paths,
    'validation': x_val_paths
  }
  return partition, labels

In [ ]:
# preprocessing the dataset

def normalize_img(img):
    norm_img = (img - img.min()) / (img.max() - img.min())
    return norm_img

def preprocess_image(img_path, horizontal_flip=False):
  image = cv2.imread(img_path)
  image = cv2.resize(image, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
  # image = image[:, 21:149].astype("float")
  image = image.astype("float")
  image = normalize_img(image)

  if horizontal_flip:
    image = cv2.flip(image, 1)
  return image

def preprocess_depth_map(depth_map_path, horizontal_flip=False):
  depth_map = cv2.imread(depth_map_path)
  depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)
  depth_map = cv2.resize(depth_map, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
  # depth_map = depth_map[:, 21:149].astype("float")
  depth_map = depth_map.astype("float")
  depth_map = normalize_img(depth_map)

  if horizontal_flip:
    depth_map = cv2.flip(depth_map, 1)

  depth_map = np.reshape(depth_map, (depth_map.shape[0], depth_map.shape[1], 1))
  return depth_map

In [ ]:
# data generator

from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
  def __init__(self, list_IDs, labels, batch_size=16, dim=(128,128), n_channels=3, shuffle=True, pred=False):
    self.dim = dim
    self.batch_size = batch_size
    self.labels = labels
    self.list_IDs = list_IDs
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.pred = pred
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    list_IDs_temp = [self.list_IDs[k] for k in indexes]
    if self.pred:
      X = self.__data_generation(list_IDs_temp)
      return X
    X, y = self.__data_generation(list_IDs_temp)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, self.dim[0], self.dim[1],self.n_channels))

    if not self.pred:
      y = np.empty((self.batch_size, self.dim[0], self.dim[1], 1))

      for i, ID in enumerate(list_IDs_temp):
        res = random.choice([True, False])
        X[i,] = preprocess_image(ID, res)
        y[i,] = preprocess_depth_map(self.labels[ID], res)
      # return X, y
      return tf.cast(X, tf.float32), tf.cast(y, tf.float32)
    else:
      for i, ID in enumerate(list_IDs_temp):
        res = random.choice([True, False])
        X[i,] = preprocess_image(ID, res)
      return X

In [ ]:
partition, labels = load_train_paths(TRAIN_PATH)

In [ ]:
print(len(partition['train']), len(partition['validation']))

22437 9617


In [ ]:
training_generator = DataGenerator(list_IDs=partition['train'], labels=labels, batch_size=16, dim=(HEIGHT, WIDTH), n_channels=3, shuffle=True, pred=False)
validation_generator = DataGenerator(list_IDs=partition['validation'], labels=labels, batch_size=16, dim=(HEIGHT, WIDTH), n_channels=3, shuffle=True, pred=False)

In [ ]:
def poly_decay(epoch):
  maxEpochs = EPOCHS
  baseLR = INIT_LR
  power = 1.0
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  return alpha

opt = Adam(learning_rate=INIT_LR, amsgrad=True)
callbacks = [LearningRateScheduler(poly_decay)]

# custom loss
def depth_loss(y_true, y_pred):
  w1, w2, w3 = 1.0, 1.0, 0.3

  l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

  dy_true, dx_true = tf.image.image_gradients(y_true)
  dy_pred, dx_pred = tf.image.image_gradients(y_pred)
  l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

  l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, 1.0)) * 0.5, 0, 1)

  return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

#custom soft accuracy
def depth_acc(y_true, y_pred):
  return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the callback to save only the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/CV/tubes_IR/ir_best.keras',  # File path where the best model will be saved
    monitor='val_loss',  # Metric to monitor for improvement
    save_best_only=True,  # Save only the best model
    save_weights_only=False,  # Save the entire model (not just weights)
    mode='min',  # 'min' means we want to minimize the monitored metric (val_loss)
    verbose=1  # Show a message when the best model is saved
)

In [ ]:
# Option 1: Load the entire model
# model = tf.keras.models.load_model('/content/drive/MyDrive/Computer Vision/tubes/best_model.keras', custom_objects={'depth_loss': depth_loss, 'depth_acc': depth_acc})

In [ ]:
model.compile(optimizer=opt, loss=depth_loss, metrics=[depth_acc])

In [ ]:
r = model.fit(training_generator, validation_data=validation_generator, epochs=15, callbacks=[callbacks, checkpoint_callback])

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1402/1402 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - depth_acc: 0.8237 - loss: 0.1856
Epoch 1: val_loss improved from inf to 0.11619, saving model to /content/drive/MyDrive/CV/tubes_IR/ir_best.keras
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 691s 372ms/step - depth_acc: 0.8237 - loss: 0.1856 - val_depth_acc: 0.9140 - val_loss: 0.1162 - learning_rate: 1.0000e-04
Epoch 2/15
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - depth_acc: 0.9160 - loss: 0.1104
Epoch 2: val_loss improved from 0.11619 to 0.10148, saving model to /content/drive/MyDrive/CV/tubes_IR/ir_best.keras
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 516s 367ms/step - depth_acc: 0.9160 - loss: 0.1104 - val_depth_acc: 0.9238 - val_loss: 0.1015 - learning_rate: 9.3333e-05
Epoch 3/15
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - depth_acc: 0.9281 - loss: 0.0970
Epoch 3: val_loss improved from 0.10148 to 0.09872, saving model to /content/drive/MyDrive/CV/tubes_IR/ir_best.keras
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 514s 366ms/step - depth_acc: 0.9281 - loss: 0.0970 - v

In [ ]:
model.save("iresnetv2_modify_15.keras")

In [ ]:
from tensorflow.keras.models import load_model
# r = model.load_weights("/content/drive/MyDrive/Computer Vision/tubes/best_model.keras")
r = tf.keras.models.load_model('/content/drive/MyDrive/Computer Vision/tubes/best_model.keras', custom_objects={'depth_loss': depth_loss, 'depth_acc': depth_acc})
# r.compile(optimizer=opt, loss=depth_loss, metrics=[depth_acc])

ValueError: File not found: filepath=/content/drive/MyDrive/Computer Vision/tubes/best_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# model2 = load_model("/kaggle/input/resnet_50_depth/keras/default/1/resnet50_unet.keras")

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# e = np.linspace(1, EPOCHS, num=EPOCHS)

plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')

plt.plot(r.history['depth_acc'], label='acc')
plt.plot(r.history['val_depth_acc'], label='val_acc')

plt.legend()
plt.show()

In [ ]:
img_dm_pairs = read_csv('./data/nyu2_test.csv')
labels = {i: j for i,j in img_dm_pairs}
test_paths = [i for i,j in img_dm_pairs]
print(len(test_paths))
partition = {'test': test_paths}

In [ ]:
img_dm_pairs = read_csv('./data/nyu2_train.csv')
labels2 = {i: j for i,j in img_dm_pairs}
train_paths2 = [i for i,j in img_dm_pairs]
print(len(train_paths2))
partition2 = {'train': train_paths2}

In [ ]:
x_test = np.empty((len(test_paths), HEIGHT, WIDTH, 3))
y_test = np.empty((len(test_paths), HEIGHT, WIDTH, 1))

for i, ID in enumerate(partition['test'][:]):
  x_test[i, ] = preprocess_image(ID, horizontal_flip=False)
  y_test[i, ] = preprocess_depth_map(labels[ID], horizontal_flip=False)

In [ ]:
print(model.evaluate(x_test, y_test))

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmzhang0526/nyu-depth-v2-labeled")

print("Path to dataset files:", path)

In [ ]:
# egen data generate
import h5py
import numpy as np

# Load the .mat file (adjust the file path as needed)
with h5py.File('/root/.cache/kagglehub/datasets/rmzhang0526/nyu-depth-v2-labeled/versions/1/nyu_depth_v2_labeled.mat', 'r') as file:
    # Access the data
    rgb_images = np.array(file['images'])  # Shape: (N, 3, 480, 640)
    depth_maps = np.array(file['depths'])  # Shape: (N, 480, 640)

# Transpose rgb_images to shape (N, 480, 640, 3) for better compatibility
rgb_images = rgb_images.transpose(0, 3, 2, 1)
depth_maps = depth_maps.transpose(0, 2, 1)

# Eigen test indices (replace with actual indices)
#984
# eigen_test_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984])  # Replace with the actual indices you want to use
# 654
eigen_test_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653])
# 100
# eigen_test_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

eigen_test_rgb = rgb_images[eigen_test_indices]
eigen_test_depth = depth_maps[eigen_test_indices]

# Define crop values (Eigen et al. crop values)
crop = [20, 460, 24, 616]  # [y_start, y_end, x_start, x_end]

# Crop the depth maps based on Eigen crop values
eigen_test_depth_cropped = eigen_test_depth[:, crop[0]:crop[1]+1, crop[2]:crop[3]+1]

# Save the generated arrays to .npy files
np.save('eigen_test_rgb.npy', eigen_test_rgb)
np.save('eigen_test_depth.npy', eigen_test_depth_cropped)
np.save('eigen_test_crop.npy', crop)

print("Eigen RGB, depth, and crop files generated and saved.")

In [ ]:
import numpy as np
from sklearn.metrics import r2_score

r2_scores = []

# Load test data
def load_test_data():
    print('Loading test data...', end='')

    rgb = np.load('eigen_test_rgb.npy')
    depth = np.load('eigen_test_depth.npy')
    crop = np.load('eigen_test_crop.npy')
    print('Test data loaded.\n')

    return rgb, depth, crop

def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=6):
    # Support multiple RGBs, one RGB image, even grayscale

    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    images = tf.image.resize(images, [240, 320])
    predictions = model.predict(images, batch_size=batch_size)

    return predictions

def scale_up(scale, images):
    from skimage.transform import resize
    scaled = []
    for i in range(len(images)):
        img = images[i]
        output_shape = (scale * img.shape[0], scale * img.shape[1])
        scaled.append( resize(img, output_shape, order=1, preserve_range=True, mode='reflect', anti_aliasing=True ) )
    return np.stack(scaled)

def evaluate(model, rgb, depth, crop, batch_size=6):
    def compute_errors(gt, pred):
        pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]))

        gt_pool = np.expand_dims(np.expand_dims(gt, axis=0), axis=-1)  # Shape: (1, H, W, 1)
        pred_pool = np.expand_dims(np.expand_dims(pred, axis=0), axis=-1)  # Shape: (1, H, W, 1)
        gt_pool = AveragePooling2D(pool_size=(7, 7), strides=(2, 2))(gt_pool)
        pred_pool = AveragePooling2D(pool_size=(7, 7), strides=(2, 2))(pred_pool)
        gt_pool = np.squeeze(gt_pool)  # Shape: (H_new, W_new)
        pred_pool = np.squeeze(pred_pool)  # Shape: (H_new, W_new)
        # Flatten the depth maps
        ground_truth_flat = gt_pool.flatten()
        predicted_flat = pred_pool.flatten()

        # Compute R^2 score for the current image and store it
        r2 = r2_score(ground_truth_flat, predicted_flat)
        r2_scores.append(r2)
        thresh = np.maximum((gt / pred), (pred / gt))

        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        abs_rel = np.mean((np.abs(gt - pred) / gt))

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())

        log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()

        return a1, a2, a3, abs_rel, rmse, log_10

    depth_scores = np.zeros((6, len(rgb))) # six metrics

    bs = batch_size

    for i in range(len(rgb)//bs):    #len(rgb)//bs
        x = rgb[(i)*bs:(i+1)*bs,:,:,:]
        # Compute results
        true_y = depth[(i)*bs:(i+1)*bs,:,:]
        pred_y = scale_up(2, predict(model, x/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0
        # Test time augmentation: mirror image estimate
        pred_y_flip = scale_up(2, predict(model, x[...,::-1,:]/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0

        pred_y = pred_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]

        # Compute errors per image in batch
        for j in range(len(true_y)):
#             errors = compute_errors(true_y[j], (0.5 * pred_y[j]) + (0.5 * np.fliplr(pred_y_flip[j])))
            errors = compute_errors((true_y[j]), (0.75 * pred_y[j]))
            for k in range(len(errors)):
                depth_scores[k][(i*bs)+j] = errors[k]

    e = depth_scores.mean(axis=1)

    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))


rgb, depth, crop = load_test_data()
evaluate(model, rgb, depth, crop)

In [ ]:
# Filter out negative R^2 scores
valid_r2_scores = [score for score in r2_scores if 0 <= score <= 1]

# Compute the average of valid R^2 scores
avg_r2_score = np.mean(valid_r2_scores)

# Plot the histogram for valid R^2 scores
plt.figure(figsize=(8, 6))
plt.hist(valid_r2_scores, bins=20, color='skyblue', edgecolor='black')
plt.axvline(avg_r2_score, color='red', linestyle='--', label=f'Avg R² = {avg_r2_score:.4f}')

# Add labels and title
plt.xlabel('R² Score', fontsize=14)
plt.ylabel('# of Test Image', fontsize=14)
plt.legend()
plt.grid(True)

# Set x-axis limits between 0 and 1
plt.xlim(0, 1)
plt.savefig('r2_scores_distribution_f01', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()

# Print the average R^2 value of valid scores
print(f'Average Valid R² Value: {avg_r2_score:.4f}')


In [ ]:
import matplotlib.pyplot as plt

# Plot individual R² scores as dots
plt.figure(figsize=(8, 6))
plt.scatter(range(len(r2_scores)), r2_scores, color='skyblue', edgecolor='black', label='R² Scores', alpha=0.7)

# Plot the average R² value as a straight line
plt.plot(range(len(r2_scores)), [avg_r2_score] * len(r2_scores), color='red', linestyle='--', label=f'Avg R² = {avg_r2_score:.4f}')

# Add labels and title
# plt.title('R² Scores Across Test Images', fontsize=16)
plt.xlabel('Image Index', fontsize=14)
plt.ylabel('R² Score', fontsize=14)
plt.legend()
plt.grid(True)

# Set y-axis limits between 0 and 1 for better visualization
plt.ylim(0, 1)
plt.savefig('r2_scores_distribution_dot_straight_final_0_1', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
#preprocess data
plt.figure(figsize=(20, 2))
for i in range(8):
  path = partition['test'][i]
  label_path = labels[path]

  img1 = preprocess_image(path, horizontal_flip=False)
  plt.subplot(1,8,i+1)
  plt.axis("off")
  plt.imshow(img1, cmap=plt.get_cmap('gray'))
plt.suptitle('Preprocessed images of the training set', fontsize=16)
plt.show()

In [ ]:
#preprocess depth map
#preprocess data
plt.figure(figsize=(20, 2))
for i in range(8):
  path = partition['test'][i]
  label_path = labels[path]


  img1 = preprocess_depth_map(label_path, horizontal_flip=False)
  plt.subplot(1,8,i+1)
  plt.axis("off")
  plt.imshow(img1, cmap=plt.get_cmap('gray'))
plt.suptitle('Preprocessed depth map images of the training set', fontsize=16)
plt.show()

In [ ]:
#preprocess depth map
#preprocess data
plt.figure(figsize=(20, 2))
for i in range(8):
  path = partition['test'][i]
  label_path = labels[path]


  img1 = preprocess_depth_map(label_path, horizontal_flip=False)
  plt.subplot(1,8,i+1)
  plt.axis("off")
  plt.imshow(img1, cmap=plt.get_cmap('inferno_r'))
plt.suptitle('Preprocessed depth map images of the training set', fontsize=16)
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
# feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('mixed_5b').output)
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block35_1_mixed').output)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(features[0, :, :, 0], cmap='inferno_r')
plt.suptitle('Feature Maps Visualization of second concatenation of IRv2', fontsize=16)
plt.show()

In [ ]:
#feature extraction of the pre-processed data
import numpy as np
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

# Load pre-trained Inception-ResNet-v2 model with weights pre-trained on ImageNet
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('mixed_6a').output)

# Load and preprocess an example image
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(features[0, :, :, 0], cmap='inferno_r')
plt.suptitle('Feature Maps Visualization of first concatenation of IRv2', fontsize=16)
plt.show()




In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block17_20_ac').output)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(features[0, :, :, 0], cmap='inferno_r')
plt.suptitle('Feature Maps Visualization of second concatenation of IRv2', fontsize=16)
plt.show()

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block8_5_mixed').output)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(features[0, :, :, 0], cmap='inferno_r')
plt.suptitle('Feature Maps Visualization of third concatenation of IRv2', fontsize=16)
plt.show()

In [ ]:
from keras.layers import UpSampling2D
from keras.models import Model
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

# Load the base model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Example layer to use before upsampling
x = base_model.get_layer('block8_5_mixed').output

# Add an upsampling layer
upsampled = UpSampling2D(size=(2, 2))(x)  # Adjust the size as needed

# Create a new model with upsampling
feature_extraction_model = Model(inputs=base_model.input, outputs=upsampled)

plt.figure(figsize=(20, 8))  # Adjust figure size as needed

for i in range(8):
    img_path = partition['test'][i]  # Path to the image
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the upsampled feature representation (output) for the input image
    upsampled_features = feature_extraction_model.predict(img_array)

    # Display the upsampled feature map
    plt.subplot(2, 8, i + 9)
    plt.axis('off')
    # Normalize feature maps to [0, 1] for display
    feature_map = upsampled_features[0, :, :, 0]
    feature_map -= feature_map.min()  # Shift values to start from 0
    feature_map /= feature_map.max()  # Scale to [0, 1]
    plt.imshow(feature_map, cmap='inferno_r')  # Apply 'viridis' colormap
    plt.title('Upsampled')

plt.suptitle('Original and Upsampled Images', fontsize=16)
plt.show()


In [ ]:
from keras.layers import UpSampling2D
from keras.models import Model
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

# Load the base model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Choose a layer for feature extraction
layer_name = 'block8_5_mixed'  # Example layer
x = base_model.get_layer(name=layer_name).output

# Add an upsampling layer
upsampled = UpSampling2D(size=(2, 2))(x)  # Adjust the size as needed

# Create models for feature extraction and upsampling
feature_extraction_model = Model(inputs=base_model.input, outputs=x)
upsampling_model = Model(inputs=base_model.input, outputs=upsampled)

plt.figure(figsize=(24, 12))  # Adjust figure size as needed

for i in range(8):
    img_path = partition['test'][i]  # Path to the image
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the downsampled feature representation
    downsampled_features = feature_extraction_model.predict(img_array)

    # Get the upsampled feature map
    upsampled_features = upsampling_model.predict(img_array)

    # Display the original image
    plt.subplot(3, 8, i + 1)
    plt.axis('off')
    plt.imshow(img_array[0] / 255.0)
    plt.title(f'Original\n{img_array.shape[1]}x{img_array.shape[2]}')

    # Display the downsampled feature map
    plt.subplot(3, 8, i + 9)
    plt.axis('off')
    downsampled_map = downsampled_features[0, :, :, 0]
    downsampled_map -= downsampled_map.min()  # Normalize for display
    downsampled_map /= downsampled_map.max()
    plt.imshow(downsampled_map, cmap='inferno_r')
    plt.title(f'Downsampled\n{downsampled_features.shape[1]}x{downsampled_features.shape[2]}')

    # Display the upsampled feature map
    plt.subplot(3, 8, i + 17)
    plt.axis('off')
    upsampled_map = upsampled_features[0, :, :, 0]
    upsampled_map -= upsampled_map.min()  # Normalize for display
    upsampled_map /= upsampled_map.max()
    plt.imshow(upsampled_map, cmap='inferno_r')
    plt.title(f'Upsampled\n{upsampled_features.shape[1]}x{upsampled_features.shape[2]}')

plt.suptitle('Original, Downsampled, and Upsampled Images', fontsize=16)
plt.show()


In [ ]:
from keras.layers import UpSampling2D
from keras.models import Model
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

# Load the base model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Define the layers for downsampling and upsampling
x = base_model.get_layer('block17_20_ac').output

# Create an upsampling layer
upsampled = UpSampling2D(size=(2, 2))(x)  # Adjust the size as needed

# Create models for downsampling and upsampling
downsampling_model = Model(inputs=base_model.input, outputs=x)
upsampling_model = Model(inputs=base_model.input, outputs=upsampled)

plt.figure(figsize=(20, 12))

for i in range(8):
    img_path = partition['test'][i]  # Path to the image
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get downsampled feature representation
    downsampled_features = downsampling_model.predict(img_array)

    # Get upsampled feature map
    upsampled_features = upsampling_model.predict(img_array)


    # Display the downsampled feature map
    plt.subplot(3, 8, i + 9)
    plt.axis('off')
    downsampled_map = downsampled_features[0, :, :, 0]
    downsampled_map -= downsampled_map.min()  # Normalize for display
    downsampled_map /= downsampled_map.max()
    plt.imshow(downsampled_map, cmap='inferno_r')
    plt.title(f'Downsampled\n{downsampled_features.shape[1]}x{downsampled_features.shape[2]}')

    # Display the upsampled feature map
    plt.subplot(3, 8, i + 17)
    plt.axis('off')
    upsampled_map = upsampled_features[0, :, :, 0]
    upsampled_map -= upsampled_map.min()  # Normalize for display
    upsampled_map /= upsampled_map.max()
    plt.imshow(upsampled_map, cmap='inferno_r')
    plt.title(f'Upsampled\n{upsampled_features.shape[1]}x{upsampled_features.shape[2]}')

plt.suptitle('Original, Downsampled, and Upsampled Images', fontsize=16)
plt.show()


In [ ]:
#upsample
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.layers import UpSampling2D

# Load pre-trained Inception-ResNet-v2 model with weights pre-trained on ImageNet
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('mixed_6a').output)

# Add an upsampling layer
upsampled = UpSampling2D(size=(2, 2))(feature_extraction_model.output)

# Create a new model with upsampling
upsampling_model = Model(inputs=base_model.input, outputs=upsampled)

plt.figure(figsize=(24, 12))  # Adjust figure size as needed

for i in range(8):
    img_path = partition['test'][i]  # Path to the image
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the downsampled feature representation
    downsampled_features = feature_extraction_model.predict(img_array)

    # Get the upsampled feature map
    upsampled_features = upsampling_model.predict(img_array)

    # Display the original image
    plt.subplot(3, 8, i + 1)
    plt.axis('off')
    plt.imshow(img_array[0] / 255.0)
    plt.title(f'Original\n{img_array.shape[1]}x{img_array.shape[2]}')

    # Display the downsampled feature map
    plt.subplot(3, 8, i + 9)
    plt.axis('off')
    downsampled_map = downsampled_features[0, :, :, 0]
    downsampled_map -= downsampled_map.min()  # Normalize for display
    downsampled_map /= downsampled_map.max()
    plt.imshow(downsampled_map, cmap='inferno_r')
    plt.title(f'Downsampled\n{downsampled_features.shape[1]}x{downsampled_features.shape[2]}')

    # Display the upsampled feature map
    plt.subplot(3, 8, i + 17)
    plt.axis('off')
    upsampled_map = upsampled_features[0, :, :, 0]
    upsampled_map -= upsampled_map.min()  # Normalize for display
    upsampled_map /= upsampled_map.max()
    plt.imshow(upsampled_map, cmap='inferno_r')
    plt.title(f'Upsampled\n{upsampled_features.shape[1]}x{upsampled_features.shape[2]}')

plt.suptitle('Original, Downsampled, and Upsampled Images', fontsize=16)
plt.show()


In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv_7b').output)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(features[0, :, :, 0], cmap='inferno_r')
plt.suptitle('Feature Maps Visualization of fourth concatenation of IRv2', fontsize=16)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.layers import UpSampling2D

# Load pre-trained Inception-ResNet-v2 model with weights pre-trained on ImageNet
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Choose the layer for feature extraction
layer_name = 'conv_7b'
x = base_model.get_layer(name=layer_name).output

# Add an upsampling layer
upsampled = UpSampling2D(size=(2, 2))(x)  # Adjust the size as needed

# Create models for feature extraction and upsampling
feature_extraction_model = Model(inputs=base_model.input, outputs=x)
upsampling_model = Model(inputs=base_model.input, outputs=upsampled)

plt.figure(figsize=(24, 12))  # Adjust figure size as needed

for i in range(8):
    img_path = partition['test'][i]  # Path to the image
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the downsampled feature representation
    downsampled_features = feature_extraction_model.predict(img_array)

    # Get the upsampled feature map
    upsampled_features = upsampling_model.predict(img_array)

    # Display the original image
    plt.subplot(3, 8, i + 1)
    plt.axis('off')
    plt.imshow(img_array[0] / 255.0)
    plt.title(f'Original\n{img_array.shape[1]}x{img_array.shape[2]}')

    # Display the downsampled feature map
    plt.subplot(3, 8, i + 9)
    plt.axis('off')
    downsampled_map = downsampled_features[0, :, :, 0]
    downsampled_map -= downsampled_map.min()  # Normalize for display
    downsampled_map /= downsampled_map.max()
    plt.imshow(downsampled_map, cmap='inferno_r')
    plt.title(f'Downsampled\n{downsampled_features.shape[1]}x{downsampled_features.shape[2]}')

    # Display the upsampled feature map
    plt.subplot(3, 8, i + 17)
    plt.axis('off')
    upsampled_map = upsampled_features[0, :, :, 0]
    upsampled_map -= upsampled_map.min()  # Normalize for display
    upsampled_map /= upsampled_map.max()
    plt.imshow(upsampled_map, cmap='inferno_r')
    plt.title(f'Upsampled\n{upsampled_features.shape[1]}x{upsampled_features.shape[2]}')

plt.suptitle('Original, Downsampled, and Upsampled Images', fontsize=16)
plt.show()


In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
preds = model.predict(x_test)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    # Print the shape of the input and output tensors
#     print("Input shape:", base_model.input_shape)
#     print("Output shape:", features.shape)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(preds[i], cmap='gray')
plt.suptitle('Output', fontsize=16)
plt.show()

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Create a model for feature extraction
preds = model.predict(x_test)
plt.figure(figsize=(20, 2))
for i in range(8):
    path = partition['test'][i]
#   label_path = labels[path]
    img_path = path
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the feature representation (output) for the input image
    features = feature_extraction_model.predict(img_array)

    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(preds[i], cmap='inferno_r')
plt.suptitle('Output', fontsize=16)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

preds = model.predict(x_test)

for i in range(8):
  path = partition['test'][i]
  label_path = labels[path]
  pred = preds[i]
  pred = np.squeeze(pred, axis=-1)

  plt.subplot(1,3,1)
  plt.axis("off")
  plt.title("Prediction")
  plt.imshow(pred, cmap=plt.get_cmap('inferno_r'))

  plt.subplot(1,3,2)
  plt.axis("off")
  plt.title("GT")
  img = preprocess_depth_map(label_path, horizontal_flip=False)
  img = np.squeeze(img, axis=-1)
  plt.imshow(img, cmap=plt.get_cmap('inferno_r'))

  plt.subplot(1,3,3)
  plt.axis("off")
  plt.title("Original RGB image")
  img1 = preprocess_image(path, horizontal_flip=False)
  plt.imshow(img1)

  plt.show()